In [1]:
import os
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import chromadb
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [2]:
model_name = "BAAI/bge-base-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/opt/homebrew/Caskroom/miniconda/base/envs/baby311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
# from langchain.llms import LlamaCpp
llm = LlamaCpp(
    model_path="/Users/Shared/Models/llama-2-13b.Q5_K_M.gguf",
    n_gpu_layers=99,
    n_batch=512,
    n_ctx=2048,
    # f16_kv=True,  
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /Users/Shared/Models/llama-2-13b.Q5_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q5_K     [  5120,  512

In [4]:
page_files = sorted(os.listdir('../data/2_html_to_txt/'))
# page_ids = list(map(lambda x: x.split('.')[0], page_files))

pages = []
for page_file in page_files:
    page = open(f'../data/2_html_to_txt/{page_file}', 'r').read()
    pages.append(page)

# whole_dataset = "\n".join(pages)

In [ ]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
# texts = text_splitter.split_text(whole_dataset)

In [5]:
texts = pages
len(texts)

660

In [6]:
embeddings = hf.embed_documents(texts)

In [7]:
client = chromadb.PersistentClient(path="chroma/cibc_pages")
# checks whether collection exists

collection = client.get_or_create_collection(name="cibc_BGE_txt_raw", metadata={"hnsw:space": "cosine"})

In [ ]:
metadatas = []
for page_name in page_files:
    tags = page_name.split(".")[0].split("_")
    tags = list(filter(lambda x: len(x) > 1, tags))
    tags_dict = {f"tag_{i}":tags[i] for i in range(len(tags))}
    metadatas.append(tags_dict)

In [8]:
page_ids = [f"doc_{i}" for i in range(len(texts))]

In [9]:
collection.add(
    documents=texts,
    embeddings=embeddings,
    # metadatas=metadatas,
    ids=page_ids,
)

In [10]:
question = "What are the credit cards offered by CIBC and what are some questions that I should ask my client to decide which one is the best fit for them?"
q_embedding = hf.embed_query(question)
response = collection.query(
    query_embeddings=q_embedding,
    n_results=10,
    include=["documents", "distances"],
    #where={"metadata_field": "is_equal_to_this"},
    #where_document={"$contains":"sock"}
)

In [11]:
response

{'ids': [['doc_107',
   'doc_92',
   'doc_74',
   'doc_47',
   'doc_78',
   'doc_112',
   'doc_116',
   'doc_127',
   'doc_640',
   'doc_80']],
 'distances': [[0.12018513679504395,
   0.12211096286773682,
   0.1238521933555603,
   0.12639671564102173,
   0.1303526759147644,
   0.13055121898651123,
   0.13096976280212402,
   0.1325356364250183,
   0.1326488113543196,
   0.13302689790725708]],
 'metadatas': None,
 'embeddings': None,
 'documents': [['\nCredit Cards\n\n  * Compare All CIBC Credit Cards \n  * Credit Card Finder \n  * Rewards Credit Cards \n  * Manage My Card \n  * Tips and Advice \n  * Costco Cards \n  * How to read your credit card statement \n\n#  COMPARE and find the best cibc CREDIT CARDS for you  \n\nAdd up to 4 credit cards and find a card that meets your needs.\n\n##  Using our compare tool is easy. Let’s get started\n\n  1. Add up to 4 CIBC credit cards   \n\n  2. Review each card and compare features like rewards, annual fees, interest rates and welcome offers \n 

In [12]:
qa_template = """You are tasked with answering a single question given a context. If you are uncertain about the answer, simply respond 'I do not know.'\n\nContext: {CONTEXT}\n\nQuestion: {QUESTION}"""

multiple_docs_template = """Document: {DOC_NAME}\n Content:\n\n```plaintext\n{CONTENT}\n```"""

In [13]:
context = []
for id, doc in zip(response["ids"][0], response["documents"][0]):
    context.append(multiple_docs_template.format(DOC_NAME=id, CONTENT=doc))

In [14]:
context

['Document: doc_107\n Content:\n\n```plaintext\n\nCredit Cards\n\n  * Compare All CIBC Credit Cards \n  * Credit Card Finder \n  * Rewards Credit Cards \n  * Manage My Card \n  * Tips and Advice \n  * Costco Cards \n  * How to read your credit card statement \n\n#  COMPARE and find the best cibc CREDIT CARDS for you  \n\nAdd up to 4 credit cards and find a card that meets your needs.\n\n##  Using our compare tool is easy. Let’s get started\n\n  1. Add up to 4 CIBC credit cards   \n\n  2. Review each card and compare features like rewards, annual fees, interest rates and welcome offers \n  3. Choose the card that’s most beneficial for you \n  4. Apply \n\nSummary  summary  headline  Key features  features-id  feature-1-id\nfeatures_1_headline  features_1_copy  feature-2-id  features_2_headline\nfeatures_2_copy  feature-3-id  features_3_headline  features_3_copy  Annual\nfee and interest rate(s)  rates-fees-id  interest-rate-id  purchaseratelabel\npurchaserate  cashratelabel  cashrate  A

In [17]:
for i in range(len(context)):
    final_prompt =  qa_template.format(CONTEXT=context[1], QUESTION=question)
    #print(get_completion(final_prompt, temperature=0, max_tokens=256))
    llm(final_prompt, temperature=0.5, max_tokens=256)



```


Answer


llama_print_timings:        load time = 14463.76 ms
llama_print_timings:      sample time =     6.57 ms /     5 runs   (    1.31 ms per token,   761.27 tokens per second)
llama_print_timings: prompt eval time = 16224.83 ms /   849 tokens (   19.11 ms per token,    52.33 tokens per second)
llama_print_timings:        eval time =   184.93 ms /     4 runs   (   46.23 ms per token,    21.63 tokens per second)
llama_print_timings:       total time = 16429.83 ms
Llama.generate: prefix-match hit


:
```

Answer:


llama_print_timings:        load time = 14463.76 ms
llama_print_timings:      sample time =    13.59 ms /     8 runs   (    1.70 ms per token,   588.62 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   370.94 ms /     8 runs   (   46.37 ms per token,    21.57 tokens per second)
llama_print_timings:       total time =   413.59 ms
Llama.generate: prefix-match hit


 The credit cards offered by CIBC include cash back, travel rewards, and no annual fee. Some questions that can help a client decide which card is best for them include their budget, spending habits, and long-term goals.

```

Answer

...


llama_print_timings:        load time = 14463.76 ms
llama_print_timings:      sample time =     2.91 ms /     4 runs   (    0.73 ms per token,  1373.15 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   254.25 ms /     4 runs   (   63.56 ms per token,    15.73 tokens per second)
llama_print_timings:       total time =   263.15 ms
Llama.generate: prefix-match hit


Answer:


```plaintext

There are several types of credit cards offered by CIBC. Some of the most popular ones include cash back credit cards, travel rewards credit cards, and low interest rate credit cards. When choosing a credit card, it's important to consider factors such as annual fees, interest rates, rewards programs, and perks.

Some questions that you can ask your client to help them decide which credit card is the best fit for them include:

- What are their spending habits? Do they frequently use credit cards for everyday purchases or do they only use them occasionally?
- What type of rewards program do they prefer? Some people may prefer cash back while others may prefer travel points.
- Do they have a specific budget in mind when choosing a credit card? This will help you narrow down the options based on their spending habits and goals.
- Are there any specific perks or features that are important to them? For example, some people may want a low interest rate while others 


llama_print_timings:        load time = 14463.76 ms
llama_print_timings:      sample time =   187.75 ms /   256 runs   (    0.73 ms per token,  1363.50 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time = 16824.57 ms /   256 runs   (   65.72 ms per token,    15.22 tokens per second)
llama_print_timings:       total time = 17409.45 ms
Llama.generate: prefix-match hit


: CIBC offers various types of credit cards, including cash back, travel rewards, and low interest. Some questions a client may want to consider when deciding which card is right for them include their spending habits, budget, and any specific features they are looking for in a card. It's important to compare the benefits and fees of different cards before making a decision.
```



llama_print_timings:        load time = 14463.76 ms
llama_print_timings:      sample time =    63.59 ms /    87 runs   (    0.73 ms per token,  1368.03 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =  5599.14 ms /    87 runs   (   64.36 ms per token,    15.54 tokens per second)
llama_print_timings:       total time =  5788.67 ms
